In [ ]:
%load_ext autoreload
%autoreload 2

import utils
import os
from string import Template

# -----------------------------------
# Paths

base_result_path = "Data"
result_path = os.path.join(base_result_path, "LLM")

if not os.path.exists(result_path):
    os.mkdir(result_path)

data_path = "/share/temp/students/heilts/Hiwi/data"

personas = ["Persona A", "Persona B", "Persona C"]

# -----------------------------------
# Global Parameters

COMPUTE_NEW_QUERIES = True
NUMBER_OF_QUERIES_TO_GENERATE = 5


In [ ]:

task_type = "queries"


system_prompt_template =  Template("Pretend to be a person with the following attributes: $attributes. " + 
                "Based on these attributes, choose other likes, dislikes and personality traits that align with them." + 
                "Only respond as this person in a way that aligns with the personality, including speaking and writing style and the words you use.")

issue = "immigration"

query_quality = "Make sure to make the queries look and sound like real queries people would enter into YouTube in a writing style that matches your personality."

user_prompt_template = Template("First introduce yourself, who are you? What likes, dislikes and interests do you have?" +  
                                "Now imagine you are alone at home and you want to watch some videos about immigration that reflect your sentiment." +  
                                "Name $n queries you would search for on YouTube." + query_quality) 

personas = {
    "persona_1" : "[white, middle aged man, has a high school education, has a full-time job, very negative opinion immigration]", 
    "persona_2" : "[young woman, academic background, heavily online, lives in the city, positive opinion about immigration]",
    "persona_3" : "[middle aged man, has basic education, has a working-class background, neutral opinion about immigration]",
}

if COMPUTE_NEW_QUERIES:
    for persona in personas:
        
        print("-"*40)
        utils.flush()

        pipe = utils.load_pipeline(use="mistral")

        task_prompt = system_prompt_template.substitute(attributes=personas[persona])
        input_prompt = user_prompt_template.substitute(n=NUMBER_OF_QUERIES_TO_GENERATE, issue=issue)

        prompt = f"<s> [INST] {task_prompt} [/INST] Model answer </s> [INST] {input_prompt} [/INST]"

        outputs = pipe(
            prompt, 
            max_new_tokens=256, 
            do_sample=True, 
            num_return_sequences=1,
            temperature=0.75, top_k=50, top_p=0.95,
            )

        print(outputs[0]["generated_text"])

        del pipe